In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import ijson
import urllib.parse

import sqlite3

# Commit the changes and close the connection
conn = sqlite3.connect('cards_database.db')



In [7]:
def process_data(soup):
    # Initialize a list to hold card data
    cards_data = []

    # Find all card rows
    for row in soup.find_all('div', class_=('row','amtAndPrice')):
        if not row:
            continue
        card_info = {}
        
        try:    # Extract card name
            name_tag = row.find('a', class_='item-name-bold')
            if name_tag:
                card_info['Name'] = name_tag.text.strip()
            
            # Extract brand
            brand_tag = row.find(text='Brand:').find_next('p')
            if brand_tag:
                card_info['Brand'] = brand_tag.text.strip()
            
            # Extract set
            set_tag = row.find(text='Set:').find_next('p')
            if set_tag:
                card_info['Set'] = set_tag.text.strip()
            
            # Extract language
            language_tag = row.find(text='Language:').find_next('p')
            if language_tag:
                card_info['Language'] = language_tag.text.strip()
            
            # Extract finish
            finish_tag = row.find(text='Finish:').find_next('p')
            if finish_tag:
                card_info['Finish'] = finish_tag.text.strip()
            
            # Extract rarity
            rarity_tag = row.find(text='Rarity:').find_next('p')
            if rarity_tag:
                card_info['Rarity'] = rarity_tag.text.strip()
            
            # Extract status
            status_tag = row.find(text='Status:').find_next('p')
            if status_tag:
                card_info['Status'] = status_tag.text.strip()
            
            stock_details = {}
            stock_qty_tag = row.find('span', class_='styleQty')
            if stock_qty_tag:
                card_info['Stock Quantity'] = stock_qty_tag.text.strip()
            else:
                card_info['Stock Quantity'] = 0

            
            stock_avail_tag = row.find('span', class_='styleQtyAvailText')
            if stock_avail_tag:
                card_info['Stock Availability'] = stock_avail_tag.text.strip()
            else:
                card_info['Stock Availability'] = 0

            stock_price_tag = row.find('span', class_='stylePrice')
            if stock_price_tag:
                card_info['Stock Price'] = stock_price_tag.text.strip()
            else:
                card_info['Stock Price'] = 0
            #  the card info to the list
            cards_data.append(card_info)
        except Exception as e:
            continue
    # Create a DataFrame
    df = pd.DataFrame(cards_data)
    return df.fillna(0) 

In [8]:
def web_scraper(url):
    # Send a GET request to the specified URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        df = process_data(soup)
        # Extract and return the title of the page
        return df
    else:
        return f"Error: Unable to retrieve page, status code {response.status_code}"


In [9]:
def insert_db(conn, data):
    # Connect to the SQLite database (or create it if it doesn't exist)

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()


    # Assuming 'data' is a DataFrame with the relevant columns
    for index, row in data.iterrows():
        cursor.execute('''
            INSERT INTO cards (name, brand, set_name, language, finish, rarity, status, stock_quantity, stock_availability, stock_price)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (row['Name'], row['Brand'], row['Set'], row['Language'], row['Finish'], row['Rarity'], row['Status'], row['Stock Quantity'], row['Stock Availability'], row['Stock Price']))


In [10]:

with open('all-cards-20241118101926.json', 'r') as f:
    # Use ijson's basic iterator to parse the file incrementally
    objects = ijson.items(f, 'item')  # Replace 'item' with the correct key if needed
    for obj in objects:
        # Process each object one at a time
        print(obj['name'])
        # for pair in zip(obj.keys(),obj.values()):
        #     print(pair)
        parsed_txt = urllib.parse.quote(obj['name'], safe='')
        data = web_scraper(f'https://www.classroomtgc.com/Product/Index2?item_name={parsed_txt}')
        display(data)
        insert_db(conn, data)

conn.commit()
conn.close()


Forest


C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  brand_tag = row.find(text='Brand:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:22: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  set_tag = row.find(text='Set:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  language_tag = row.find(text='Language:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  finish_tag = row.find(text='Finish:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:37: DeprecationWarning: The 'text' argument to find()

,Name,Brand,Set,Language,Finish,Rarity,Status,Stock Quantity,Stock Availability,Stock Price
0,Forest,Magic: the Gathering,Duel Decks: Nissa vs. Ob Nixilis,English,nonfoil,Common,Active,0,0,MYR 1
1,Forest,Magic: the Gathering,Core Set 2020,English,foil,Common,Active,0,0,MYR 2
2,Forest,Magic: the Gathering,Core Set 2020,English,nonfoil,Common,Active,0,0,MYR 1
3,Forest,Magic: the Gathering,Jumpstart,English,nonfoil,Common,Active,0,0,MYR 1
4,Forest,Magic: the Gathering,War of the Spark,English,foil,Common,Active,0,0,MYR 2
5,Forest,Magic: the Gathering,Magic 2011,English,nonfoil,Common,Active,0,0,MYR 1
6,Forest,Magic: the Gathering,Duel Decks: Mind vs. Might,English,nonfoil,Common,Active,0,0,MYR 1
7,Forest,Magic: the Gathering,Magic 2011,English,foil,Common,Active,0,0,MYR 4
8,Forest,Magic: the Gathering,Magic Origins,English,nonfoil,Common,Active,0,0,MYR 1
9,Forest,Magic: the Gathering,Duel Decks: Mind vs. Might,English,nonfoil,Common,Active,0,0,MYR 1


Fury Sliver


C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  brand_tag = row.find(text='Brand:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:22: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  set_tag = row.find(text='Set:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  language_tag = row.find(text='Language:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  finish_tag = row.find(text='Finish:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:37: DeprecationWarning: The 'text' argument to find()

...

,Name,Brand,Set,Language,Finish,Rarity,Status,Stock Quantity,Stock Availability,Stock Price
0,Ichthyomorphosis,Magic: the Gathering,Theros Beyond Death,English,foil,Common,Active,8,available,MYR 2
1,Ichthyomorphosis,Magic: the Gathering,Theros Beyond Death,English,nonfoil,Common,Active,8,available,MYR 1


Mountain


C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  brand_tag = row.find(text='Brand:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:22: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  set_tag = row.find(text='Set:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  language_tag = row.find(text='Language:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  finish_tag = row.find(text='Finish:').find_next('p')
C:\Users\Owner\AppData\Local\Temp\ipykernel_11884\263800967.py:37: DeprecationWarning: The 'text' argument to find()

,Name,Brand,Set,Language,Finish,Rarity,Status,Stock Quantity,Stock Availability,Stock Price
0,Mountain,Magic: the Gathering,Unlimited Edition,English,nonfoil,Common,Active,0,0,MYR 12
1,Mountain (Retro),Magic: the Gathering,Portal Three Kingdoms,English,nonfoil,Common,Active,0,0,MYR 47
2,Mountain (Showcase),Magic: the Gathering,Core Set 2021,English,nonfoil,Common,Active,0,0,MYR 3
3,Mountain,Magic: the Gathering,Magic Premiere Shop 2005,English,foil,Common,Active,0,0,MYR 50
4,Mountain (Showcase),Magic: the Gathering,Core Set 2021,English,foil,Common,Active,0,0,MYR 4
5,Mountain (Retro),Magic: the Gathering,Seventh Edition,English,nonfoil,Common,Active,0,0,MYR 2
6,Mountain,Magic: the Gathering,Starter Commander Decks,English,nonfoil,Common,Active,0,0,MYR 1
7,Mountain,Magic: the Gathering,Foreign Black Border,English,nonfoil,Common,Active,0,0,MYR 18
8,Mountain,Magic: the Gathering,Commander Legends: Battle for Baldur's Gate,English,nonfoil,Common,Active,0,0,MYR 1
9,Mountain,Magic: the Gathering,Secret Lair Drop,English,foil,Rare,Active,0,0,MYR 18


Mortify


In [5]:
# Query the database to retrieve data
conn = sqlite3.connect('cards_database.db')

cursor = conn.cursor()

query_result = cursor.execute('SELECT * FROM cards').fetchall()
query_result

[]

In [4]:
cursor.execute('DELETE FROM cards')
conn.commit()
conn.close()
